In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *

In [ ]:
import numpy as np

In [ ]:
width = 0.1
thickness = width
cross_section = [thickness, width]

In [ ]:
input_path = '../../data/hemisphere.json'
target_mesh_path = '../../data/target_meshes/hemisphere.obj'
# input_path = '../../data/TB.json'
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
width = 2*input_data['arm_plate_edge_offset']
thickness = width * 0.5 # 1.5 mm # FIX from mm to meters everywhere
cross_section = [thickness, width]

In [ ]:
import numpy.linalg as la

In [ ]:
with open("correspondence_points.obj", 'w') as f:
    for pt in input_data['correspondence']:
        if (la.norm(pt) > 0):
            f.write("v {} {} {}\n".format(pt[0], pt[1], pt[2]))

### Initialization

In [ ]:
# curr_um = umbrella_mesh.UmbrellaMesh(io)
curr_um = umbrella_mesh.UmbrellaMesh(target_mesh_path, io)

In [ ]:
curr_um.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))

In [ ]:
curr_um.energy(UmbrellaEnergyType.Full)

### Pin Rigid Motion



In [ ]:
use_pin = False

In [ ]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

### Equilibrium solve

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-2
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 500
OPTS.verboseNonPosDef = False

In [ ]:
rod_colors = []
for ri in range(curr_um.numSegments()):
    rod_colors.append(np.ones(curr_um.segment(ri).rod.numVertices()) * ri)

In [ ]:
import mesh
target_mesh = mesh.Mesh(target_mesh_path)
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
# view = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
# view.update(scalarField = rod_colors)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)
view.show()

In [ ]:
dof = curr_um.getDoFs()
for i in range(curr_um.numJoints()):
    if (curr_um.joint(i).jointType() == umbrella_mesh.JointType.X):
        dof[curr_um.dofOffsetForJoint(i) + 6] = 1e-3
curr_um.setDoFs(dof)

In [ ]:
view.update()
view.showScalarField(rod_colors)

In [ ]:
angles = []
def eqm_callback(prob, i):
    angles.append(curr_um.getDoFs()[curr_um.jointAngleDoFIndices()])
    if (i % 10 == 0):
        view.update()
        view.showScalarField(rod_colors)

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 0.1
# curr_um.deploymentForceType = umbrella_mesh.DeploymentForceType.Constant
curr_um.targetDeploymentHeight = thickness * 2
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 1000
curr_um.setHoldClosestPointsFixed(False)
curr_um.scaleInputPosWeights(0.5)

In [ ]:
curr_um.energyElastic(), curr_um.energyDeployment(), curr_um.energyRepulsion(), curr_um.energyAttraction()

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars)
view.showScalarField(rod_colors)